https://book.naver.com/category/index.nhn?cate_code=100010&list_type=list&tab=recommend


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from bs4 import BeautifulSoup
from selenium import webdriver

import requests
import urllib.request
import urllib.error
import urllib.parse
import re
import json
#import math
#import datetime
#import os
#import sys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_naver_blog_text_v4(url):
    
    def get_compiled_text_from_soup(soup, tag):
        contents = []
        contentsArea = soup.select(tag)
        if contentsArea:
            # real url의 div#postViewArea로부터 콘텐츠 텍스트 불러오기
            for content in contentsArea:
                # html 태그를 제거한 뒤 텍스트만 contents 리스트에 저장
                contents = [str(content.get_text()) for content in soup.select(tag)]
                
        return contents

        # 리스트 안의 문자열을 모두 합쳐서 반환하고 함수 종료
        # 문자열 없었다면 empty string 반환하고 함수 종료
        return ' '.join(contents)
    
    def create_soup_from_url(url):
        get_blog_post_content_code = requests.get(url)
        get_blog_post_content_text = get_blog_post_content_code.text
        return BeautifulSoup(get_blog_post_content_text, 'lxml')
        
    real_blog_post_url = ''
    contents = []
    fake_url = str(url)
    
    # 주소값 체크 - string으로 바꾼 값이 nan이면 None 반환
    if fake_url == 'nan':
        print('nan', end=' ')
        return None

    try:
        # fake url로부터 BeautifulSoup 생성
        #time.sleep(sleep_time)
        get_blog_post_content_soup = create_soup_from_url(fake_url)

        # fake url의 mainFrame 속성값으로부터 real url 생성
        real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')

        # real url로부터 BeautifulSoup 생성            
        get_real_blog_post_content_soup = create_soup_from_url(real_blog_post_url)            
        contents = get_compiled_text_from_soup(get_real_blog_post_content_soup,
                                              'div#postViewArea')

        if contents:
            print('/', end=' ')
            return contents
        else:
            raise IndexError

    except:           
        try:
            # 만약 위에서 함수가 종료되지 않았다면 hidden tag값을 생성
            uid = real_blog_post_url.split('logNo=')[1].split('&amp')[0].split('&redirect')[0]
            hidden_tag = 'div#post-view' + uid
            contents = get_compiled_text_from_soup(get_real_blog_post_content_soup, hidden_tag)

            # 만약 contents 리스트에 뭔가 담겨 있다면
            if contents:
                # 리스트 안의 문자열을 모두 합쳐서 반환하고 함수 종료
                print('/', end=' ')
                return contents
            else:
                raise IndexError

        except:
            old_post_tag = 'div.rvw_cnt'
            old_soup = create_soup_from_url(fake_url)
            contents = get_compiled_text_from_soup(old_soup, old_post_tag)

            if contents:
                print('/', end=' ')
                return contents

            else:
                #print(f'url:{fake_url} / real blog post url:{real_blog_post_url}')
                print('*', end=' ')
                return None


### Crawling

In [92]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome(
    '/usr/local/bin/chromedriver', options=options)
url = 'https://book.naver.com/category/index.nhn?cate_code=100010&tab=recommend&list_type=list&sort_type=salecount&page=1'

titles = []
reviews = []
urls = []
intros = []
intos = []
blog_urls = []

try:
    for l in range(17, 33):
        ### 최상위 페이지
        print('current page:', l, end='/')
        print('collected:', len(titles), len(urls), len(blog_urls), end='/')
        
        current_page_url = "https://book.naver.com/category/index.nhn?cate_code=100010&tab=recommend&list_type=list&sort_type=salecount&page={}".format(l)
        url = current_page_url
        driver.get(url)

        ### 페이지 상에 존재하는 책 리스트 (페이지당 20개)
        for i in range(1,21):
            print('book num', i, end='/')
                       
            ### 책 제목 경로 받아오기
            novel_title_xpath = f'//*[@id="book_title_{i-1}"]/a[1]'.format(l-1)
            
            try:
                driver.get(url)
                
                ### 책 링크 저장 > book_url
                book_hyperlink = driver.find_element_by_xpath(novel_title_xpath)
                book_url = book_hyperlink.get_attribute('href')
                
                ### 책 제목 저장 > title
                title = book_hyperlink.text
                
                ### url에 포함된 id 저장 > book_id
                book_id = book_url.split('=')[1]
                
                ### 소설 타이틀 클릭
                driver.find_element_by_xpath(novel_title_xpath).click()
                time.sleep(1)
                
                try:
                    ### 책 웹페이지 진입
                    ### 책 소개 텍스트 저장 > book_intro_text
                    book_intro = driver.find_element_by_xpath('//*[@id="bookIntroContent"]/p')
                    time.sleep(.5)
                    book_intro_text = book_intro.text

                    
                    ### '책 속으로' 섹션 존재하는지 검색, 있으면 저장 > book_into_text
                    book_into_text = ''
                    for divnum in range(1, 14):
                        form = f'//*[@id="content"]/div[{divnum}]'
                        
                        try:
                            book_into_head = driver.find_element_by_xpath(form+'/h3').text
                            if book_into_head in ['책 속으로', '책속으로']:
                                book_into_text = driver.find_element_by_xpath(form).text
                                break
                        except:
                            pass
                    
                    try:
                        ### 책 리뷰 페이지 진입
                        book_review_p_url = f'https://book.naver.com/bookdb/review.nhn?bid={book_id}'
                        driver.get(book_review_p_url)
                        time.sleep(.5)

                        ### 리뷰 페이지에 리뷰는 1페이지당 10개씩
                        element = WebDriverWait(driver, 3).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[1]/span'))
                        )
                        leng = element.text[1:-2]
                        
                        ### 리뷰 총 갯수                        
                        leng = leng.replace(',', '')
                        leng = int(leng)
                        book_review_len = leng
                        
                        ### 최대 리뷰 수집갯수 = 책 1권당 100건
                        if book_review_len > 100:
                            book_review_len = 100
                            
                        ### loop 돌아야 할 리뷰 페이지의 갯수
                        review_page_length = book_review_len // 10 + 1
                            
                        try:
                            for j in range(1, review_page_length+1):
                                
                                ### j번째 리뷰 페이지 주소 접근
                                book_review_p_url_loop = book_review_p_url + '&page=' + str(j)
                                driver.get(book_review_p_url_loop)
                                time.sleep(1)
                                
    
                                ### 현재 리뷰 페이지에서 확인해야 할 리뷰 숫자를 체크
                                ### 첫번째 리뷰 페이지의 리뷰 갯수
                                fpage_review_length = (10 if book_review_len >= 10 else book_review_len)

                                ### 마지막 리뷰 페이지의 리뷰 갯수
                                lpage_review_length = (0 if book_review_len == fpage_review_length\
                                                       else (book_review_len - (book_review_len//10)*10))
                                
                                ### 총 리뷰 페이지가 1개라면 fpage_review_length만큼 loop
                                ### 현재 리뷰 페이지가 마지막 리뷰 페이지라면 lpage_review_length만큼 loop
                                ### 총 리뷰 페이지가 1개도 아니고, 마지막 페이지도 아니면 10만큼 loop
                                num_reviews_to_loop = (fpage_review_length if review_page_length == 1\
                                                      else lpage_review_length if j == review_page_length\
                                                      else 10)
                                
                                for k in range(1, num_reviews_to_loop+1):
                                    try:
                                        review_title_link_path = f'//*[@id="reviewList"]/li[{k}]/dl/dt/a'
                                        
                                        try:
                                            element = WebDriverWait(driver, 3).until(
                                                EC.presence_of_element_located((By.XPATH, review_title_link_path))
                                            )
                                            ### 블로그 리뷰 url, 타이틀 가져와서 각각 변수 저장
                                            blog_review_url = element.get_attribute('href')
                                            blog_review_title = element.text
                                            time.sleep(.5)
                                            
                                            titles.append(title) #책 제목 리스트 저장
                                            urls.append(book_url) #책 소개 url 리스트 저장
                                            intros.append(book_intro_text) # 책 소개 리스트 저장
                                            intos.append(book_into_text) # 책 속으로 리스트 저장
                                            blog_urls.append(blog_review_url)
                                            time.sleep(0.5)

                                        except:
                                            print('fail-0')
 
                                    except:
                                        print('fail-1')
                                
                        except:
                            print('fail-2')
                    except:
                        print('fail-3')
                except:
                    print('fail-4')
            except:
                print('fail-5')
except:
    print('except1')

finally:
    driver.close()


current page: 17/collected: 0 0 0/book num 1/book num 2/book num 3/book num 4/book num 5/book num 6/book num 7/book num 8/book num 9/book num 10/book num 11/book num 12/book num 13/book num 14/book num 15/book num 16/book num 17/book num 18/book num 19/book num 20/current page: 18/collected: 1152 1152 1152/book num 1/book num 2/book num 3/book num 4/book num 5/book num 6/book num 7/book num 8/book num 9/book num 10/book num 11/book num 12/fail-4
book num 13/book num 14/book num 15/book num 16/book num 17/book num 18/book num 19/book num 20/current page: 19/collected: 2286 2286 2286/book num 1/book num 2/book num 3/book num 4/book num 5/book num 6/book num 7/book num 8/book num 9/book num 10/book num 11/book num 12/book num 13/book num 14/book num 15/book num 16/book num 17/fail-4
book num 18/book num 19/book num 20/current page: 20/collected: 3584 3584 3584/book num 1/book num 2/book num 3/book num 4/book num 5/book num 6/book num 7/book num 8/book num 9/book num 10/book num 11/book nu

In [95]:
print(len(titles), len(urls), len(intros), len(intos), len(reviews), len(blog_urls))

(11562, 11562, 11562, 11562, 0, 11562)

In [111]:
df_book = pd.DataFrame({"title":titles, "book_url":urls, "intro":intros,
                          "into":intos, "blog_url":blog_urls})

In [114]:
df_book.to_csv('./team_csv/book_review_17_32.csv', encoding='utf-8-sig')

In [12]:
#df_book = pd.read_csv('./team_csv/book_review_17_32.csv', index_col = 0)

# 리뷰 URL > 리뷰 주소로 변환하기

In [13]:
df_book['review'] = [get_naver_blog_text_v4(url) for url in df_book['blog_url']]

/ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / * / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / * / / / / / * / / / / / / / / / / / / / / / / / / / / / nan / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * * / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / * * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / * / / / / * / / * * * * * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 

/ / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / * / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / * / / / / / / / / / / / / / * / / * / / / * * / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / * / / / * * * / / / / / / / * / / / * / / / * / * / / * / / / / * / / / / / / / / / / / * / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / * / / / / * / / / / / / / / / * / / / / / / * / / / / / / / / / * / / * / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / * / / / / / / / / / / / * * * * / * / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 

/ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / * / / / / * / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / * / / / / / / / * / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / 

In [15]:
review_49 = pd.read_csv('./team_csv/book_review_49.csv', index_col=0)
review_49['review'] = [get_naver_blog_text_v4(url) for url in review_49['blog_url']]
review_49.to_csv('./team_csv/book_review_33_49.csv', encoding='utf-8-sig')

/ / / / / / / / / / / * / / / * / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * * * / / * / / / / / * / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / * / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / * / / / / / / * / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / * / / * / / / / / / / / / / / / / / / / * / / / / / / / / * * / / / / / / / / / / / / / / / / / / / * * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / / / / / / / * / / / / / / / / / / / / / * / * / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / * / * / / * / / / / * / * / / / / / * * / / / / / / / / / / / / / / / / * / / / / / / / 

/ / / / / / / / / 

# `get_naver_blog_text()` 테스트
* epoch = 1000
    * 손실률 계산
    * loop time 계산

In [7]:
blog_urls = pd.read_csv('./team_csv/book_review_17_32.csv', index_col=0)['blog_url']

In [7]:
start = time.time()

rvs_converted = []
rvs_converted_fail = []
for i, bu in enumerate(blog_urls):
    if (i % 999 == 0) and (i > 0):
        end = time.time()
        print(f'time elapsed for {i}th loop:', end - start)
        start = time.time()
        break
        
    try:
        converted = get_naver_blog_text_v3(bu)
        if not converted: ## return == None
            rvs_converted_fail.append(bu)
            print(i, bu)
            
        rvs_converted.append(converted)
    
    except:
        rvs_converted.append(None)
        rvs_converted_fail.append(bu)
        print(i, bu)
        
end = time.time()
print('time elapsed:', end - start)

40 http://blog.naver.com/yamamodo/150068237339
43 http://blog.naver.com/phchung/130067741456
62 http://blog.naver.com/joym1224/40088161337
138 http://blog.naver.com/wownsk/30067798339
144 http://blog.naver.com/jib1208/87331955
150 http://blog.naver.com/amamon/60089194532
172 nan
208 http://blog.naver.com/alistata/140105423351
209 http://blog.naver.com/samsoon07/20099105709
237 http://blog.naver.com/yhd1653/90065182023
263 http://cafe.naver.com/discussbook/879
270 http://blog.naver.com/jehee999/30067819640
271 http://blog.naver.com/kangwsjjang/20057617891
304 http://blog.naver.com/sodamy37/30067792894
315 http://blog.naver.com/minl22/40088130759
337 http://blog.naver.com/khj19891009/100087078031
342 http://blog.naver.com/greta79/10068040522
345 http://blog.naver.com/19870525/130067740189
346 http://blog.naver.com/puppy143/110067548834
347 http://blog.naver.com/krispinaa/150068145228
348 http://blog.naver.com/teevyya/10068145327
349 http://blog.naver.com/michael0122/30067981537
415 http:

In [8]:
len(rvs_converted_fail)

35

In [9]:
len(rvs_converted)

997